Install required libraries

In [ ]:
!pip install wandb
!pip install -U scikit-learn

     |████████████████████████████████| 2.1MB 5.7MB/s 
     |████████████████████████████████| 133kB 18.1MB/s 
     |████████████████████████████████| 102kB 7.1MB/s 
     |████████████████████████████████| 163kB 18.7MB/s 
     |████████████████████████████████| 71kB 6.0MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=025eb2c7acefce0161c31540271fb07e172ae7fcca11bb8ecfe6d122e454bf87
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=2e657f63e40aa73eabad20485d638761971c06266d5be230ce9fb929673b2195
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
     |████████████████████████████████| 22.3MB 4.7MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import libraries

In [ ]:
import os
import sys
import torch
import torchvision
from torchvision import transforms, datasets
import numpy as np
import sklearn
from sklearn.neighbors import KNeighborsClassifier
# import wandb #Import this to collab
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import torchvision.utils as vutils
import torch.nn as nn
import torch.optim as optim
#from config import config
import matplotlib.animation as animation
from IPython.display import HTML
from sklearn.metrics import accuracy_score
# import torchgan
# from torchgan.layers import SpectralNorm2d

In [ ]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only. If you want to enable GPU, please to go Edit > Notebook Settings > Hardware Accelerator and select GPU.")

Load images

In [ ]:
# Import from our Shared Google Drive
# Set directories for training and testing datasets

digits = range(1,11)

# Images are 32 x 32
img_size = 32
image_size = 32

# Create transforms
train_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])
test_transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

main_dir = "/content/drive/Shareddrives/EECS 504 Shared Drive/Fake Images/"

# Load all 128 images
# Create datasets
generated_images = datasets.ImageFolder(main_dir, transform=train_transform)
# test_img = datasets.ImageFolder(test_dir, transform=test_transform) 

root_dir = "/content/drive/Shareddrives/EECS 504 Shared Drive/Datasets/SVHN Dataset/"

#train_dir = root_dir + "train/"
#train_img = datasets.ImageFolder(train_dir, transform=train_transform) 

test_dir = root_dir + "test/"
test_img = datasets.ImageFolder(test_dir, transform=test_transform) 

In [ ]:
# print("train_img type   :",type(generated_images))
# print("train_img length :",len(generated_images))
# # print("test_img length :",len(test_img))
# print("train_img classes:",generated_images.classes)
# print("train_img[0] type:",type(generated_images[0]))
# print("train_img[0][0] t:",type(generated_images[0][0]))
# print("train_img[0][1] t:",type(generated_images[0][1]))
# print("train_img[0][0] s:",generated_images[0][0].size())
# print("train_img[0][1]  :",generated_images[0][1])

# n_labels = len(generated_images.classes)

print("train_img type   :",type(generated_images))
print("train_img length :",len(generated_images))
# print("test_img length :",len(test_img))
print("train_img classes:",generated_images.classes)
print("train_img[0] type:",type(generated_images[0]))
print("train_img[0][0] t:",type(generated_images[0][0]))
print("train_img[0][1] t:",type(generated_images[0][1]))
print("train_img[0][0] s:",generated_images[0][0].size())
print("train_img[0][1]  :",generated_images[0][1])

n_labels = len(generated_images.classes)

train_img type   : <class 'torchvision.datasets.folder.ImageFolder'>
train_img length : 23006
train_img classes: ['1', '10', '2', '3', '4', '5', '6', '7', '8', '9']
train_img[0] type: <class 'tuple'>
train_img[0][0] t: <class 'torch.Tensor'>
train_img[0][1] t: <class 'int'>
train_img[0][0] s: torch.Size([3, 32, 32])
train_img[0][1]  : 0


# Classification (K Nearest Neighbors)





Be sure to run everything before and including "Loading Data" part

Image processing

In [ ]:
def knn_image_proc(data_img, m = None):
  '''
  Image flattening and normalization for kNN
  Input:
    data_img: images and labels
  Output:
    x_train: processed images (flattened to be vectors and normalized)
    y: labels
  '''
  n_train = len(data_img)

  x_raw = []

  y = []

  for i in range(n_train):
    # x_i = train_img[i][0].reshape(-1)
    x_i = np.array(data_img[i][0].reshape(-1))
    x_raw.append(x_i)
    y.append(data_img[i][1])

    if i%1000 == 0:
      print('Has read ' + str(i) + ' out of ' + str(n_train) + ' images')

  # Normalize to zero mean so covariance is just matrix multiplication
  if m is None: 
    m = np.mean(x_raw, axis=0, keepdims=True)
  x = x_raw - m 

  return x, y,  m

Reduction

In [ ]:
def compute_basis(data, n=300):
  """ 
  Computing basis for image reduction
  Inputs:
      data: multi-dimensional array of size number of images * number of pixels. Each row is a flatten image.
      n: number of modes to be kept
  Output:
      eigenvectors: basis
  """
  
  eigenvectors = None

  sig = np.matmul(np.transpose(data),data)
  _, eigenvectors = np.linalg.eig(sig)
  eigenvectors = eigenvectors[:, range(n)]

  return eigenvectors

kNN

In [ ]:
# # Self defined kNN function
# def knn_predict(eig_train_x, eig_test_x, y_train, y_test, k=1):

#   """Implement the KNN algorithm. The output should be a vector containing your predictions"""

#   predictions = np.zeros_like(y_test)
    
#   N, D = eig_train_x.shape
#   for i in range(len(y_test)):
#     d = np.ones((N,))
#     for j in range(N):
#       dx = eig_train_x[j,:] - eig_test_x[i,:]
#       d[j] = np.linalg.norm(dx)

#     x_in = np.argsort(d)
#     neb_in = x_in[:k]
#     neb_y = y_train[neb_in]

#     counts = np.bincount(neb_y)
#     predictions[i] = np.argmax(counts)

#   return predictions

kNN over the fake images

In [15]:
k = 3 # k nearest neighbors

if_POD = True # Do we do POD?

# Image flattening and normalization for the original training data set
x_train, y_train, mean_train = knn_image_proc(generated_images)

# Image reduction
if if_POD:
  eigenvectors = compute_basis(x_train, n=300)
else:
  _, n_dim  = x_train.shape
  eigenvectors = np.identity(n_dim)

x_train_reduced = np.matmul(x_train, eigenvectors)

# kNN training
neigh = KNeighborsClassifier(n_neighbors = k)
neigh.fit(x_train_reduced, y_train)

Has read 0 out of 23006 images
Has read 1000 out of 23006 images
Has read 2000 out of 23006 images
Has read 3000 out of 23006 images
Has read 4000 out of 23006 images
Has read 5000 out of 23006 images
Has read 6000 out of 23006 images
Has read 7000 out of 23006 images
Has read 8000 out of 23006 images
Has read 9000 out of 23006 images
Has read 10000 out of 23006 images
Has read 11000 out of 23006 images
Has read 12000 out of 23006 images
Has read 13000 out of 23006 images
Has read 14000 out of 23006 images
Has read 15000 out of 23006 images
Has read 16000 out of 23006 images
Has read 17000 out of 23006 images
Has read 18000 out of 23006 images
Has read 19000 out of 23006 images
Has read 20000 out of 23006 images
Has read 21000 out of 23006 images
Has read 22000 out of 23006 images
Has read 23000 out of 23006 images


KNeighborsClassifier(n_neighbors=3)

In [16]:
# Image flattening and normalization for the original test data set
x_test, y_test, _ = knn_image_proc(test_img, mean_train)

x_test_reduced = np.matmul(x_test, eigenvectors)

# kNN prediction
y_test_kNN = neigh.predict(x_test_reduced)

## Evalution
accuracy = np.sum(y_test_kNN == np.array(y_test))/y_test_kNN.size
print('The accuracy of kNN with generated images is: ', accuracy*100, "%")

Has read 0 out of 25982 images
Has read 1000 out of 25982 images
Has read 2000 out of 25982 images
Has read 3000 out of 25982 images
Has read 4000 out of 25982 images
Has read 5000 out of 25982 images
Has read 6000 out of 25982 images
Has read 7000 out of 25982 images
Has read 8000 out of 25982 images
Has read 9000 out of 25982 images
Has read 10000 out of 25982 images
Has read 11000 out of 25982 images
Has read 12000 out of 25982 images
Has read 13000 out of 25982 images
Has read 14000 out of 25982 images
Has read 15000 out of 25982 images
Has read 16000 out of 25982 images
Has read 17000 out of 25982 images
Has read 18000 out of 25982 images
Has read 19000 out of 25982 images
Has read 20000 out of 25982 images
Has read 21000 out of 25982 images
Has read 22000 out of 25982 images
Has read 23000 out of 25982 images
Has read 24000 out of 25982 images
Has read 25000 out of 25982 images
The accuracy of kNN with generated images is:  21.326302825032712 %


Visualization